In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.metrics import mean_absolute_error, mean_squared_error
import dash
from dash import dcc, html
import plotly.graph_objs as go

In [12]:
df = pd.read_csv(r'C:\Users\hp\Desktop\VAFS-main\stockDATA\INFY.csv')

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)

In [14]:
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()
df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()

In [15]:
delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

In [16]:
df.bfill(inplace=True)

In [17]:
features = ['Close', 'MA_10', 'MA_50', 'EMA_10', 'RSI']
data = df[features].values

In [18]:
data = df[['Close']].values

In [19]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [20]:
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(x), np.array(y)

In [21]:
seq_length = 60
x, y = create_sequences(scaled_data, seq_length)

In [22]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [23]:
model = Sequential([
    Input(shape=(x_train.shape[1], x_train.shape[2])),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

In [24]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [25]:
model.fit(x_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 71s 16ms/step - loss: 0.0011
Epoch 2/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 66s 16ms/step - loss: 5.3780e-04
Epoch 3/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 65s 15ms/step - loss: 4.2718e-04
Epoch 4/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 67s 16ms/step - loss: 3.8873e-04
Epoch 5/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 65s 15ms/step - loss: 2.9292e-04
Epoch 6/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 65s 15ms/step - loss: 3.0854e-04
Epoch 7/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 72s 17ms/step - loss: 3.0235e-04
Epoch 8/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 73s 17ms/step - loss: 3.3489e-04
Epoch 9/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 76s 18ms/step - loss: 2.5673e-04
Epoch 10/10
4196/4196 ━━━━━━━━━━━━━━━━━━━━ 73s 17ms/step - loss: 3.1789e-04


In [26]:
predictions = model.predict(x_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [27]:
# Ensure predictions is a 2D array before concatenation
predictions = predictions.reshape(-1, 1)

# Concatenate predictions with zeros to match original feature dimensions
predictions = scaler.inverse_transform(
    np.concatenate((predictions, np.zeros((predictions.shape[0], scaled_data.shape[1] - 1))), axis=1)
)[:, 0]


In [28]:
y_test = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], scaled_data.shape[1] - 1))), axis=1)
)[:, 0]

In [29]:
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

In [30]:
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 145.47796615776699
Root Mean Squared Error: 148.24434993243167


In [31]:
df_test = df.iloc[-len(y_test):].copy()
df_test.loc[:,'Predictions'] = predictions

In [34]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("INFY Stock Price Visualization and Forecasting"),
    dcc.Graph(
        id='historical-data',
        figure={
            'data': [
                go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Actual Prices'),
                go.Scatter(x=df_test['Date'], y=df_test['Predictions'], mode='lines', name='Predicted Prices')
            ],
            'layout': go.Layout(
                title='INFY BANK Stock Prices',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'},
                hovermode='closest'
            )
        }
    ),
    html.Div([
        html.H3(f"Mean Absolute Error: {mae:.2f}"),
        html.H3(f"Root Mean Squared Error: {rmse:.2f}")
    ])
])

if __name__ == '__main__':
   app.run(debug=True)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run.